In [1]:
debug = False

import ROOT as R

if debug:
    R.ROOT.EnableImplicitMT() # only for ROOT rdf

import os
from pprint import pprint
import pandas as pd 


Welcome to JupyROOT 6.19/01


In [2]:
import PyRDF
if not debug:
    PyRDF.use("spark")
PyRDF.include_headers("headers.hh")

print(PyRDF.current_backend.supported_operations)

['Define', 'Filter', 'Histo1D', 'Histo2D', 'Histo3D', 'Profile1D', 'Profile2D', 'Profile3D', 'Count', 'Sum', 'Fill', 'Snapshot', 'Graph', 'AsNumpy']


In [3]:
# this is only for root rdf "interactive"
if debug:
    R.gInterpreter.ProcessLine(".L headers.hh")

In [4]:
import latinos_rdf as lrdf

In [9]:
version = "v5"

# input
config_dir = "Full2017v6"

# selection
#samples = ["DATA","VBS", "VBF-V", "Wjets", "singleTop", "ttbar",  "VV","DY","FakeQCD", "VVV"]
samples = ["VBS", "top", "DY", "Wjets"]
weights = ["weight_", "XSWeight"]
cut = "lowen_looseVBS"
#cut = "boost_looseVBS"

# output
output_basedir = "/eos/user/d/dmapelli/public/latino/Full2017v6" # replace the last dir with the "config" variable?
output_dir = output_basedir + "/" + cut + "/samples/" + version + "/"


import os 
os.makedirs(output_dir, exist_ok=True)

In [11]:
config = lrdf.ConfigReader(config_dir)

columns_MC = config.variables

#columns_DATA = columns
columns_MC = [ cut+"_"+c for c in columns_MC]

columns_MC = weights + columns_MC

print(columns_MC)

['weight_', 'XSWeight', 'lowen_looseVBS_Lepton_RecoSF', 'lowen_looseVBS_LepWPWeight', 'lowen_looseVBS_puWeight', 'lowen_looseVBS_TriggerEffWeight_1l', 'lowen_looseVBS_btagSF', 'lowen_looseVBS_events', 'lowen_looseVBS_Lepton_pt', 'lowen_looseVBS_Lepton_eta', 'lowen_looseVBS_Lepton_phi', 'lowen_looseVBS_Lepton_flavour', 'lowen_looseVBS_PuppiMET', 'lowen_looseVBS_PuppiMET_phi', 'lowen_looseVBS_MET_pt', 'lowen_looseVBS_recoMET_pz', 'lowen_looseVBS_nJets', 'lowen_looseVBS_N_jets_central', 'lowen_looseVBS_N_jets_forward', 'lowen_looseVBS_mjj_vbs', 'lowen_looseVBS_deltaeta_vbs', 'lowen_looseVBS_deltaphi_vbs', 'lowen_looseVBS_vbs_etaprod', 'lowen_looseVBS_vbs_0_pt', 'lowen_looseVBS_vbs_1_pt', 'lowen_looseVBS_vbs_0_eta', 'lowen_looseVBS_vbs_1_eta', 'lowen_looseVBS_vbs_0_phi', 'lowen_looseVBS_vbs_1_phi', 'lowen_looseVBS_vbs_index_0', 'lowen_looseVBS_vbs_index_1', 'lowen_looseVBS_mjj_vjet', 'lowen_looseVBS_vjet_0_pt', 'lowen_looseVBS_vjet_1_pt', 'lowen_looseVBS_vjet_0_eta', 'lowen_looseVBS_vjet_1

In [12]:
joblist = []

for sample in samples:
    if debug:
        trees, nfiles = lrdf.build_dataframe(config_dir, sample, R, "root") # ROOT RDF "interactive"
    else:
        trees, nfiles = lrdf.build_dataframe(config_dir,  sample, PyRDF , "pyrdf") # Spark
    for tree, nfile in zip(trees,nfiles):
        joblist.append((tree,nfile))
        
######## Now sort by number of files
jobslist = sorted(joblist, key=lambda v: v[1], reverse=True)


In [13]:
########
dfs = {}
for tree,_ in jobslist:
        print(tree)
        # Choose the right output columns
        if sample == "DATA": outputcols = columns_DATA
        else: outputcols = columns_MC

        df = pd.DataFrame(tree[cut].rdf_node.AsNumpy(columns=outputcols))
        print(tree.name, df.shape)
        
        if tree.name in dfs:
            df.to_pickle(output_dir + tree.name + f"_part{dfs[tree.name]+1}.pkl")
            dfs[tree.name] +=1
        else:
            df.to_pickle(output_dir + tree.name +"_part1.pkl")
            dfs[tree.name] =1
            
        

Tree: Wjets
name: supercut
parent: None
cut: (                      ( (abs(Lepton_pdgId[0])==11) && (Lepton_pt[0]>40) ) ||                      ( (abs(Lepton_pdgId[0])==13) && (Lepton_pt[0]>30) )                       )
vars: 
aliases: bVeto,bReq,bVetoSF,bReqSF,btagSF
weight: (XSWeight*puWeight*                   TriggerEffWeight_1l*                   Lepton_RecoSF[0]*                   EMTFbug_veto*Lepton_tightElectron_mvaFall17V1Iso_WP90_IdIsoSF[0]*                Lepton_tightMuon_cut_Tight_HWWW_IdIsoSF[0]*(Lepton_isTightElectron_mvaFall17V1Iso_WP90[0]>0.5 || Lepton_isTightMuon_cut_Tight_HWWW[0]>0.5)*PrefireWeight*btagSF*METFilter_MC*Lepton_genmatched[0])*((1.))
--------------------------------------------------------------------------------
name: lowen_looseVBS
parent: supercut
cut: VBS_category==1             && vbs_0_pt > 30 && vbs_1_pt > 30              && vjet_0_pt > 30 && vjet_1_pt > 30             && mjj_vbs >=300                && deltaeta_vbs >= 2              && PuppiMET_pt

top (52037, 55)
Tree: DY
name: supercut
parent: None
cut: (                      ( (abs(Lepton_pdgId[0])==11) && (Lepton_pt[0]>40) ) ||                      ( (abs(Lepton_pdgId[0])==13) && (Lepton_pt[0]>30) )                       )
vars: 
aliases: bVeto,bReq,bVetoSF,bReqSF,btagSF
weight: (XSWeight*puWeight*                   TriggerEffWeight_1l*                   Lepton_RecoSF[0]*                   EMTFbug_veto*Lepton_tightElectron_mvaFall17V1Iso_WP90_IdIsoSF[0]*                Lepton_tightMuon_cut_Tight_HWWW_IdIsoSF[0]*(Lepton_isTightElectron_mvaFall17V1Iso_WP90[0]>0.5 || Lepton_isTightMuon_cut_Tight_HWWW[0]>0.5)*PrefireWeight*btagSF*METFilter_MC*Lepton_genmatched[0]*(Sum(GenPart_pdgId == 22 && OddVec(GenPart_statusFlags) && GenPart_pt > 20.) == 0))*((1.)*(((0.632927+0.0456956*gen_ptll-0.00154485*gen_ptll*gen_ptll+2.64397e-05*gen_ptll*gen_ptll*gen_ptll-2.19374e-07*gen_ptll*gen_ptll*gen_ptll*gen_ptll+6.99751e-10*gen_ptll*gen_ptll*gen_ptll*gen_ptll*gen_ptll)*(gen_ptll>0)*(gen_ptll<100)

VBS (123132, 55)
Tree: DY
name: supercut
parent: None
cut: (                      ( (abs(Lepton_pdgId[0])==11) && (Lepton_pt[0]>40) ) ||                      ( (abs(Lepton_pdgId[0])==13) && (Lepton_pt[0]>30) )                       )
vars: 
aliases: bVeto,bReq,bVetoSF,bReqSF,btagSF
weight: (XSWeight*puWeight*                   TriggerEffWeight_1l*                   Lepton_RecoSF[0]*                   EMTFbug_veto*Lepton_tightElectron_mvaFall17V1Iso_WP90_IdIsoSF[0]*                Lepton_tightMuon_cut_Tight_HWWW_IdIsoSF[0]*(Lepton_isTightElectron_mvaFall17V1Iso_WP90[0]>0.5 || Lepton_isTightMuon_cut_Tight_HWWW[0]>0.5)*PrefireWeight*btagSF*METFilter_MC*Lepton_genmatched[0]*(Sum(GenPart_pdgId == 22 && OddVec(GenPart_statusFlags) && GenPart_pt > 20.) == 0))*((1.)*(((0.623108 + 0.0722934*gen_ptll - 0.00364918*gen_ptll*gen_ptll + 6.97227e-05*gen_ptll*gen_ptll*gen_ptll - 4.52903e-07*gen_ptll*gen_ptll*gen_ptll*gen_ptll)*(gen_ptll<45)*(gen_ptll>0) + 1*(gen_ptll>=45))*(LHE_HT<100)))
-------------

/cvmfs/sft-nightlies.cern.ch/lcg/views/dev3python3/Mon/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/PyRDF/backend/Dist.py:315: UserWarning: Number of partitions is greater than number of clustersin the filelist
Using 24 partition(s)
  filelist, self.friend_info)


DY (161, 55)
Tree: Wjets
name: supercut
parent: None
cut: (                      ( (abs(Lepton_pdgId[0])==11) && (Lepton_pt[0]>40) ) ||                      ( (abs(Lepton_pdgId[0])==13) && (Lepton_pt[0]>30) )                       )
vars: 
aliases: bVeto,bReq,bVetoSF,bReqSF,btagSF
weight: (XSWeight*puWeight*                   TriggerEffWeight_1l*                   Lepton_RecoSF[0]*                   EMTFbug_veto*Lepton_tightElectron_mvaFall17V1Iso_WP90_IdIsoSF[0]*                Lepton_tightMuon_cut_Tight_HWWW_IdIsoSF[0]*(Lepton_isTightElectron_mvaFall17V1Iso_WP90[0]>0.5 || Lepton_isTightMuon_cut_Tight_HWWW[0]>0.5)*PrefireWeight*btagSF*METFilter_MC*Lepton_genmatched[0])*((1.)*(LHE_HT < 100))
--------------------------------------------------------------------------------
name: lowen_looseVBS
parent: supercut
cut: VBS_category==1             && vbs_0_pt > 30 && vbs_1_pt > 30              && vjet_0_pt > 30 && vjet_1_pt > 30             && mjj_vbs >=300                && deltaeta_vbs >= 2

/cvmfs/sft-nightlies.cern.ch/lcg/views/dev3python3/Mon/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/PyRDF/backend/Dist.py:315: UserWarning: Number of partitions is greater than number of clustersin the filelist
Using 22 partition(s)
  filelist, self.friend_info)


Wjets (58, 55)
Tree: DY
name: supercut
parent: None
cut: (                      ( (abs(Lepton_pdgId[0])==11) && (Lepton_pt[0]>40) ) ||                      ( (abs(Lepton_pdgId[0])==13) && (Lepton_pt[0]>30) )                       )
vars: 
aliases: bVeto,bReq,bVetoSF,bReqSF,btagSF
weight: (XSWeight*puWeight*                   TriggerEffWeight_1l*                   Lepton_RecoSF[0]*                   EMTFbug_veto*Lepton_tightElectron_mvaFall17V1Iso_WP90_IdIsoSF[0]*                Lepton_tightMuon_cut_Tight_HWWW_IdIsoSF[0]*(Lepton_isTightElectron_mvaFall17V1Iso_WP90[0]>0.5 || Lepton_isTightMuon_cut_Tight_HWWW[0]>0.5)*PrefireWeight*btagSF*METFilter_MC*Lepton_genmatched[0]*(Sum(GenPart_pdgId == 22 && OddVec(GenPart_statusFlags) && GenPart_pt > 20.) == 0))*((1.)*(((0.632927+0.0456956*gen_ptll-0.00154485*gen_ptll*gen_ptll+2.64397e-05*gen_ptll*gen_ptll*gen_ptll-2.19374e-07*gen_ptll*gen_ptll*gen_ptll*gen_ptll+6.99751e-10*gen_ptll*gen_ptll*gen_ptll*gen_ptll*gen_ptll)*(gen_ptll>0)*(gen_ptll<100)+

/cvmfs/sft-nightlies.cern.ch/lcg/views/dev3python3/Mon/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/PyRDF/backend/Dist.py:315: UserWarning: Number of partitions is greater than number of clustersin the filelist
Using 7 partition(s)
  filelist, self.friend_info)


DY (0, 55)


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available
